In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#Set up Intial array
X = 17.13
K_b = 0.75
Y = 0
B_w = 30
Lambda_deg = 32 #degrees
def get_K_s_val(X,Y,B_w,K_b,Lambda_deg):
    Lambda_rad = Lambda_deg*np.pi/180
    X_bar = X/(B_w*0.5)
    Y_bar = Y/(B_w*0.5)
    r_bar = np.sqrt(X_bar**2+Y_bar**2)
    s_bar = K_b*np.tan(Lambda_rad)
    t_bar = np.sqrt((X_bar-s_bar)**2+Y_bar**2+K_b**2)
    t_0bar = np.sqrt(X_bar**2+Y_bar**2+K_b**2)
    top = 1 + (X_bar-s_bar)/(t_bar) + (X_bar*(r_bar+t_bar)*(t_0bar**2 - X_bar**2))/(r_bar*t_bar*(r_bar*t_bar+r_bar**2 -X_bar*s_bar))
    bottom = 1 + (X_bar*(r_bar**2 + t_0bar**2 -X_bar**2))/(t_0bar*r_bar**2 )
    K_s = top/bottom
    return K_s
K_s = get_K_s_val(X,Y,B_w,K_b,Lambda_deg)
print(K_s)


1.0439788293848493


In [3]:
# CL solver

def get_CL_val(W,gamma,rho,vinf,sw):
    return W*np.cos(gamma*np.pi/180)/(0.5*rho*vinf**2 *sw)
CL = get_CL_val(20500,0,0.0023769,225,300)
print(CL)


1.1357602243957488


In [4]:
#Side wash estimator
#Equation for x_d
def x_d_equation(x_hat,gamma):
    return x_hat-np.tan(gamma*np.pi/180)
#Full equation estimator given known information
def epsilon_s(es_0,es_beta,beta):
    return es_0 + es_beta*beta*(np.pi/180)

def epsilon_beta(Cl_w,Ra_w,b_w,K_v,K_b,x,y,z,gamma):
    #solve for x_hat,y_hat,z_hat,x_d
    x_hat = (2*x)/(b_w*K_b)
    y_hat = (2*y)/(b_w*K_b)
    z_hat = (2*z)/(b_w*K_b)
    x_d = x_d_equation(x_hat,gamma)
    #lead coefficient
    lead_co = -(K_v*x_d*Cl_w)/(Ra_w*K_b*np.pi**2)
    var1 = ((-2*y_hat*(z_hat-1))/(y_hat**2 + (z_hat-1)**2)**2)*(1+((x_d)/(np.sqrt(x_d**2 + y_hat**2 + (z_hat-1)**2))))
    var2 = ((2*y_hat*(z_hat+1))/(y_hat**2 + (z_hat+1)**2)**2)*(1+((x_d)/(np.sqrt(x_d**2 + y_hat**2 + (z_hat+1)**2))))
    var3 = (y_hat/(y_hat**2 + (z_hat-1)**2))* ((x_d*(z_hat-1))/((x_d**2 + y_hat**2 + (z_hat-1)**2)**(3/2)))
    var4 = (y_hat/(y_hat**2 + (z_hat+1)**2))* ((x_d*(z_hat+1))/((x_d**2 + y_hat**2 + (z_hat+1)**2)**(3/2)))
    return lead_co*(var1+var2-var3+var4)

def epsilon_beta_simplified(CL_w,RA_w,K_v,K_beta,K_b):
    return -(CL_w*K_v*K_beta)/(K_b*RA_w)

#function to get the K_beta value using the equation where it lies on axis of sym ie Z=0
def get_K_beta_val_Z_zero(K_b,b_w,gamma,x,y):
    x_hat = (2*x)/(b_w*K_b)
    y_hat = (2*y)/(b_w*K_b)
    x_d = x_d_equation(x_hat,gamma)
    var1 = ((4*y_hat*x_d)/(np.pi**2 *(y_hat**2 + 1)**2)) *(1+((x_d)/(np.sqrt(x_d**2 + y_hat**2 +1 ))))
    var2 = ((2*y_hat)/(np.pi**2 *(y_hat**2 + 1)**2)) *((x_d**2)/((x_d**2 + y_hat**2 +1)**(3/2)))
    return var1 + var2

def get_CL_v_effective(RA_eff,K_L,C_L_alpha_tilda):
    return C_L_alpha_tilda/((1+C_L_alpha_tilda/(1*np.pi*RA_eff))*(1+K_L))

def Vel_Min_Power(Cd0,Cd1,RA,e,w,Sw,rho_air):
    co_1 = np.pi*e*RA*Cd1
    co_2 = 12*np.pi*RA*Cd0
    co_3 = np.sqrt((w/Sw)/rho_air)
    return (2/np.sqrt(co_1+np.sqrt(co_1**2 + co_2)))*co_3

def Vel_Min_Drag(e,RA,Cd0,W,Sw,rho_air):
    return (np.sqrt(2)/np.sqrt(np.sqrt(np.pi*e*RA*Cd0)))*np.sqrt((W/Sw)/rho_air)

def Vel_Min_Stall(CL_max,W,Sw,rho_air):
    return np.sqrt(2/CL_max)*np.sqrt((W/Sw)/rho_air)

In [5]:
#Quiz 11 answers
print(get_CL_val(22000,0,0.000364,500*(5280/3600),1000))
print(get_K_beta_val_Z_zero(0.80,100,10,37,10))
print(epsilon_beta_simplified(0.30,10,0.98,get_K_beta_val_Z_zero(0.80,100,10,37,10),0.80))
print(get_CL_v_effective(8,0.012,2*np.pi))



0.22477522477522482
0.11884776964844729
-0.004367655534580438
4.9669449068613325


In [6]:
#giving the informaiton for A_20 of Project, baseline Glider
RA = 12
Clmax = 0
rho_air = 0.002048
Sw = 6.75
weight = 1.1002
e = 0.6854
cd0 = 0.0104
cd1 = -0.0071
CL_max = 1.3767
print(Vel_Min_Power(cd0,cd1,RA,e,weight,Sw,rho_air))
print(Vel_Min_Drag(e,RA,cd0,weight,Sw,rho_air))
print(Vel_Min_Stall(CL_max,weight,Sw,rho_air))



12.637370540208709
17.52293363050643
10.752620033522781


In [7]:
#giving the informaiton for A_20 of Project, baseline Glider
RA = 21.3333333
rho_air = 0.002048
Sw = 6.75
weight = 0.80526
e = 0.55675
cd0 = 0.01460
cd1 = -0.0101
CL_max = 1.4272 #happens at 5.7 angle of attack
print(f'Minimum Power Required Velocity: {Vel_Min_Power(cd0,cd1,RA,e,weight,Sw,rho_air)}')
print(f'Minimum Drag Velocity: {Vel_Min_Drag(e,RA,cd0,weight,Sw,rho_air)}')
print(f'Minimum Velocity to Stall: {Vel_Min_Stall(CL_max,weight,Sw,rho_air)}')



Minimum Power Required Velocity: 8.711235869344145
Minimum Drag Velocity: 12.563503944440152
Minimum Velocity to Stall: 9.03490894665049


In [8]:

#giving the informaiton for A_20 of Project, baseline Glider, 6917_new dynamic mode
RA = 23.80952
rho_air = 0.002048
Sw = 6.5625
weight = 0.78676
e = 0.48438
cd0 = 0.01530
cd1 = -0.01240
CL_max = 1.4232 #happens at 5.7 angle of attack
print(f'Minimum Power Required Velocity: {Vel_Min_Power(cd0,cd1,RA,e,weight,Sw,rho_air)}')
print(f'Minimum Drag Velocity: {Vel_Min_Drag(e,RA,cd0,weight,Sw,rho_air)}')
print(f'Minimum Velocity to Stall: {Vel_Min_Stall(CL_max,weight,Sw,rho_air)}')


Minimum Power Required Velocity: 8.444415716092584
Minimum Drag Velocity: 12.539839060343208
Minimum Velocity to Stall: 9.06992157306063


In [9]:
#Eigenvalue approximations, Quiz 26 

#Coefficients


#Get the moment information of aircraft
I_xx = 14890




#Wing Information
Wing_Area = 300
Wing_Span = 30
Wing_MeanChord = Wing_Area/Wing_Span

#################  Air Properties  ########################################

#density
Air_Density = 0.0023084

#################  Aerodynamic Properties  #################################

#CL values
CL_0 = 0.370
CL_alpha = 4.2875
CL_qbar = 3.4827
CL_alpha_hat = 0.67838

#CY values
CY_beta = 3.0112
CY_pbar = 0.7032
CY_rbar = 1.2444

#CD values
CD_0 = 0.0298
CD_alpha = 0.3239
CD_qbar = 0.7183

#Cl Values
Cl_beta = -0.287
Cl_pbar = -0.4824
Cl_rbar = 0.1274

#Cm Values
Cm_alpha = -0.7959
Cm_qbar = -4.6718
Cm_alpha_hat = -0.77143

#Cn values
Cn_beta = 0.992
Cn_pbar = -0.2311
Cn_rbar = -0.5031

#################  Solved Variables ############################################

#Gravity at location
g = 32.17 #gravity at logan elevation used for design

#Solve for the inital velocity given the launch conditions
#V_0 = np.sqrt(2*Launch_Energy*g/Weight) #Need to check if this is correct, don't do this, but we can do this later

#Get V_0 from the CL
V_0 = 400


R_gx = 0.001
R_gy = 0.003 
#10.74
R_rhox = 368.02
R_rhoy = 122.67
#10.75 
R_xx = 6.371
R_yy = 644.76
R_zz = 24.689
R_xz = 6.912



#Roll Mode

sigma_r = ((-V_0*Air_Density*Wing_Area*Wing_Span**2)/(4*I_xx))*Cl_pbar
print(f'\nDamping Rate Roll Mode: {sigma_r}')
if sigma_r > 0:
    print(f'Damping Time 99%: {np.log(0.01)/-sigma_r}')
else:
    print(f'Doubling Time: {np.log(2)/sigma_r}')

#spiral mode

sigma_spiral = (g/V_0)*((Cl_beta*Cn_rbar - Cl_rbar*Cn_beta)/(Cl_beta*Cn_pbar - Cl_pbar*Cn_beta))
print(f'\nDamping Rate Spiral Mode: {sigma_spiral}')
if sigma_spiral > 0:
    print(f'Damping Time 99%: {np.log(0.01)/-sigma_spiral}')
else:
    print(f'Doubling Time: {np.log(2)/-sigma_spiral}')

#Dutch Roll Approx
R_Ds = (Cl_beta*(R_gy*R_rhoy*R_zz-(R_rhoy-CY_rbar)*Cn_pbar) -CY_beta*Cl_rbar*Cn_pbar)/(R_rhoy*R_zz*Cl_pbar)
R_Ds = 0.0073
sigma_dr = (-V_0/Wing_Span)*(CY_beta/R_rhoy + Cn_rbar/R_zz - (Cl_rbar*Cn_pbar)/(Cl_pbar*R_zz) + (R_gy*(Cl_rbar*Cn_beta - Cl_beta*Cn_rbar))/(Cl_pbar*(Cn_beta+CY_beta*Cn_rbar/R_rhoy)) - R_xx*(R_Ds/Cl_pbar) )
Frequency_dr = (2*V_0/Wing_Span)*np.sqrt((1-CY_rbar/R_rhoy)*(Cn_beta/R_zz) + (CY_beta*Cn_rbar)/(R_rhoy*R_zz) + R_Ds -0.25*(CY_beta/R_rhoy + Cn_rbar/R_zz)**2 )


print(f'\nDamping Rate Dutch Roll Mode: {sigma_dr}')
if sigma_dr > 0:
    print(f'Damping Time 99%: {np.log(0.01)/-sigma_dr}')
else:
    print(f'Doubling Time: {np.log(2)/-sigma_dr}')
if Frequency_dr != 0:
    print(f'Damped Frequency[rad/s]: {Frequency_dr}')
    print(f'Period [s]: {2*np.pi/Frequency_dr}')
#print results
    
print(f'RDS = {R_Ds}')
print(f'Dutch Freq = {Frequency_dr}')
print(f'Dutch Damp = {sigma_dr}')


Damping Rate Roll Mode: 2.01923763062458
Damping Time 99%: 2.280647961460407

Damping Rate Spiral Mode: 0.0026582030323024034
Damping Time 99%: 1732.4373383169761

Damping Rate Dutch Roll Mode: -1.3096289064671198
Doubling Time: 0.5292699154219133
Damped Frequency[rad/s]: 5.754545881520417
Period [s]: 1.0918646643094445
RDS = 0.0073
Dutch Freq = 5.754545881520417
Dutch Damp = -1.3096289064671198
